In [2]:
import pandas as pd 
import polars as pl
import sys 
import os 
sys.path.append("..")

from settings import NB_TRANSACTIONS_PER_MONTH, PROJECT_PATH

### Version Polars 

In [38]:
transactions_per_city = pl.read_parquet(
    os.path.join(PROJECT_PATH, "transactions_par_ville.parquet")
)

transactions_per_city = transactions_per_city.with_columns(
    pl.col("departement").cast(pl.Int32),
    pl.col("mois_transaction").cast(pl.Int32),
)

In [39]:
transactions_per_city

departement,ville,id_ville,annee_transaction,mois_transaction,prix_m2_moyen,nb_transactions_mois
i32,str,i32,i32,i32,f64,u32
69,"""LYON 5EME""",385,2020,6,4001.327016,28
1,"""PREVESSIN-MOENS""",313,2021,1,4690.776739,6
83,"""SAINT-CYR-SUR-MER""",112,2022,7,6111.572834,12
85,"""TALMONT-SAINT-HILAIRE""",288,2021,2,2716.96763,6
49,"""SAUMUR""",328,2019,2,1159.580826,6
…,…,…,…,…,…,…
6,"""BEAULIEU SUR MER""",11,2021,4,6545.977778,5
92,"""ISSY-LES-MOULINEAUX""",40,2020,12,8814.489559,54
26,"""ROMANS-SUR-ISERE""",281,2018,3,1352.329575,8


In [40]:
def compute_features_price_per_m2(
    average_per_month_per_city: pl.DataFrame,
    sort_columns: list = [
        "departement",
        "ville",
        "id_ville",
        "annee_transaction",
        "mois_transaction",
    ],
    aggregation_columns :list = [
        "departement",
        "ville",
        "id_ville",
    ],
    aggregation_period="6mo",
):

    nb_months_aggregated = int(aggregation_period[0])

    '''
    Il faut absolument trier les données avant de réaliser une opération group by. 
    Sinon le calcul des rollings sera faussé car la donnée n'est pas triée chronologiquement.
    Faire un groupby ne change pas l'ordre de la donnée triée au préalable.
    '''

    average_per_month_per_city = (
        average_per_month_per_city.sort(sort_columns)
        .with_columns(
            pl.col("prix_m2_moyen")
            .shift()
            .over(aggregation_columns)
            .alias("prix_m2_moyen_mois_precedent"),
            pl.col(NB_TRANSACTIONS_PER_MONTH)
            .shift()
            .over(aggregation_columns)
            .alias("nb_transactions_mois_precedent"),
            pl.col("prix_m2_moyen")
            .rolling_mean(window_size=nb_months_aggregated)
            .over(aggregation_columns)
            .alias("prix_m2_moyen_glissant_" + aggregation_period),
            pl.col(NB_TRANSACTIONS_PER_MONTH)
            .rolling_mean(window_size=nb_months_aggregated)
            .over(aggregation_columns)
            .alias("nb_transaction_moyen_glissant_" + aggregation_period),
        )
        .filter(
            pl.all_horizontal(
                pl.col(pl.Float32, pl.Float64, pl.Int32, pl.Int64).is_not_nan()
            )
        )
    )

    return average_per_month_per_city

In [41]:
transactions_per_city = compute_features_price_per_m2(
    average_per_month_per_city=transactions_per_city,
    sort_columns = [
        "departement",
        "ville",
        "id_ville",
        "annee_transaction",
        "mois_transaction",
    ],
    aggregation_columns=[
        "departement",
        "ville",
        "id_ville",
    ],
    aggregation_period="6mo",
)

Les mois en 2018 ont disparues de la donnée après ce calcul, car elles ont été utilisées pour calculer les moyennes glissantes et que nous avons supprimé les valeurs NaNs engendrés en concéquence. 

In [29]:
transactions_per_city

departement,ville,id_ville,annee_transaction,mois_transaction,prix_m2_moyen,nb_transactions_mois,prix_m2_moyen_mois_precedent,nb_transactions_mois_precedent,prix_m2_moyen_glissant_6mo,nb_transaction_moyen_glissant_6mo
i32,str,i32,i32,i32,f64,u32,f64,u32,f64,f64
1,"""BELLEGARDE-SUR-VALSERINE""",33,2019,2,1836.642924,7,1731.895216,6,1754.182864,6.833333
1,"""BELLEGARDE-SUR-VALSERINE""",33,2019,3,1409.890396,5,1836.642924,7,1750.966969,6.666667
1,"""BELLEGARDE-SUR-VALSERINE""",33,2019,6,1474.622898,5,1409.890396,5,1697.883598,6.666667
1,"""BELLEGARDE-SUR-VALSERINE""",33,2019,7,1660.487797,9,1474.622898,5,1694.878234,6.666667
1,"""BELLEY""",34,2020,11,1569.707065,10,1205.410636,5,1391.850982,6.166667
…,…,…,…,…,…,…,…,…,…,…
974,"""SAINT-PAUL""",15,2022,3,2050.329864,5,4928.62406,5,3868.256977,5.5
974,"""SAINT-PAUL""",15,2022,4,3341.223265,6,2050.329864,5,3605.605095,5.666667
974,"""SAINT-PAUL""",15,2022,5,3778.537138,6,3341.223265,6,3659.761591,5.833333


### Version Pandas

In [71]:
transactions_per_city = pd.read_parquet(
    os.path.join(PROJECT_PATH, "transactions_par_ville.parquet")
)

In [72]:
def compute_features_price_per_m2(
    average_per_month_per_city: pd.DataFrame,
    sort_columns = [
        "departement",
        "ville",
        "id_ville",
        "annee_transaction",
        "mois_transaction",
    ],
    aggregation_columns=[
        "departement",
        "ville",
        "id_ville",
    ],
    aggregation_months=6,
):

    
    average_per_month_per_city = average_per_month_per_city.sort_values(by=sort_columns).groupby(aggregation_columns).agg(
            prix_m2_moyen_mois_precedent = pd.NamedAgg(column="prix_m2_moyen", aggfunc=lambda x: list(x.shift())),
            nb_transactions_mois_precedent = pd.NamedAgg(column=NB_TRANSACTIONS_PER_MONTH, aggfunc=lambda x: list(x.shift())),
            prix_m2_moyen_glissant_6mo = pd.NamedAgg(column="prix_m2_moyen", aggfunc=lambda x: list(x.rolling(window=aggregation_months).mean())),
            nb_transaction_moyen_glissant_x = pd.NamedAgg(column=NB_TRANSACTIONS_PER_MONTH, aggfunc=lambda x: list(x.rolling(window=6).mean())),
        )
    # On renomme la colonne pour y attribuer le vrai suffixe (dans le cas où on ne voudrait pas 6 mois)
    average_per_month_per_city = average_per_month_per_city.rename(columns={
        "nb_transaction_moyen_glissant_x" : "nb_transaction_moyen_glissant_" + str(aggregation_months),
    })
    
    # On restrucutre le DataFrame pour avoir une ligne par ville et mois 
    average_per_month_per_city = average_per_month_per_city.explode(["prix_m2_moyen_mois_precedent", "nb_transactions_mois_precedent", "prix_m2_moyen_glissant_6mo", "nb_transaction_moyen_glissant_6"])

    # On supprime les NaNs engendrés par les opérations de shift et de rolling
    average_per_month_per_city = average_per_month_per_city.dropna()

    return average_per_month_per_city

In [73]:
transactions_per_city_agg = compute_features_price_per_m2(
    average_per_month_per_city=transactions_per_city,
    sort_columns= [
        "departement",
        "ville",
        "id_ville",
        "annee_transaction",
        "mois_transaction",
    ],
    aggregation_months=6,
)

Contrairement à la combinaison with_columns() et over() de Polars, Pandas ne rappatrie pas les nouvelles colonnes crées dans le DataFrame d'origine. Il faudrait faire une jointure supplémentaire avec la donnée d'origine si nous souhaitons obtenir un format identique à celui obtenu avec Polars.

In [74]:
transactions_per_city_agg


prix_m2_moyen_mois_precedent  \
departement ville                    id_ville                                
01          BELLEGARDE-SUR-VALSERINE 33                        1731.895216   
                                     33                        1836.642924   
                                     33                        1409.890396   
                                     33                        1474.622898   
            BELLEY                   34                        1205.410636   
...                                                                    ...   
974         SAINT-PAUL               15                         4928.62406   
                                     15                        2050.329864   
                                     15                        3341.223265   
                                     15                        3778.537138   
                                     15                        5041.756649   

                                              nb_transactions_mois_precedent  \
departement ville                    id_ville                                  
01          BELLEGARDE-SUR-VALSERINE 33                                  6.0   
                                     33                                  7.0   
                                     33                                  5.0   
                                     33                                  5.0   
            BELLEY                   34                                  5.0   
...                                                                      ...   
974         SAINT-PAUL               15                                  5.0   
                                     15                                  5.0   
                                     15                                  6.0   
                                     15                                  6.0   
                                     15                                  6.0   

                                              prix_m2_moyen_glissant_6mo  \
departement ville                    id_ville                              
01          BELLEGARDE-SUR-VALSERINE 33                      1754.182864   
                                     33                      1750.966969   
                                     33                      1697.883598   
                                     33                      1694.878234   
            BELLEY                   34                      1391.850982   
...                                                                  ...   
974         SAINT-PAUL               15                      3868.256977   
                                     15                      3605.605095   
                                     15                      3659.761591   
                                     15                      3953.541502   
                                     15                      3857.354203   

                                              nb_transaction_moyen_glissant_6  
departement ville                    id_ville                                  
01          BELLEGARDE-SUR-VALSERINE 33                              6.833333  
                                     33                              6.666667  
                                     33                              6.666667  
                                     33                              6.666667  
            BELLEY                   34                              6.166667  
...                                                                       ...  
974         SAINT-PAUL               15                                   5.5  
                                     15                              5.666667  
                                     15                              5.833333  
                                     15                              5.833333  
                                     15  

### Pourquoi le scaling avant la séparation Train-Test constitue une forme de Data Leakage ?

Si vous réalisez le scaling avant de séparer vos données en jeu de Train et de Test, vous utilisez des informations du jeu de Test pour calculer les paramètres de scaling (Min et Max du jeu de données en cas de MinMaxScaler, ou la moyenne et l'écart-type en cas de StandardScaler). Cela signifie que votre modèle a accès à des informations sur les données de test lors de la phase d'entraînement, ce qui constituerait une forme de Data Leakage.

Ce Data Leakage va causer une surestimation a performance du modèle, car il aura été entraîné sur des données qui ont été conditionnés en partie par le jeu de test qu'il verra ensuite lors de la phase de test. Cela peut conduire à une baisse de performances quand vous aurez réellement de la nouvelle donnée à disposition ! 

La bonne pratique est de séparer vos données en jeu de train et de test de réaliser le scaling. Vous calculez les paramètres de scaling sur l'ensemble d'entraînement, puis vous appliquez ces paramètres à l'ensemble d'entraînement et à l'ensemble de test. Cela garantit que votre modèle n'a pas accès aux données de test lors de la phase d'entraînement.